# 🔧 Inicialización del entorno y estructura de carpetas

Esta celda configura el entorno base del proyecto:

- Añade la ruta raíz del proyecto a `sys.path`
- Instala los paquetes desde `requirements.txt` (si existe)
- Crea automáticamente las carpetas necesarias para organizar los datos:
  - `data/raw/[fuente]`
  - `data/interim/[fuente]`
  - `data/processed/[fuente]`

Estas rutas están organizadas por fuente de datos (CHIRPS, Copernicus, GEE(Google Earth Engine)) y se usarán durante todo el flujo.


In [1]:
import subprocess
import sys
from pathlib import Path
from datetime import date, timedelta

# Ruta base: carpeta raíz del proyecto (2 niveles arriba del notebook)
ruta_base = Path.cwd().parent.resolve()
sys.path.append(str(ruta_base))

# Confirmar que el path es correcto
print(f"Ruta base del proyecto agregada a sys.path: {ruta_base}")

# Crear subcarpetas organizadas por fuente
fuentes = ["chirps", "copernicus", "gee"]
etapas = ["raw", "interim", "processed"]

for etapa in etapas:
    for fuente in fuentes:
        carpeta = ruta_base / "data" / etapa / fuente
        carpeta.mkdir(parents=True, exist_ok=True)
        print(f"📁 Carpeta creada/verificada: {carpeta}")

# Definir rutas útiles para todo el notebook
RAW_CHIRPS = ruta_base / "data" / "raw" / "chirps"
RAW_COPERNICUS = ruta_base / "data" / "raw" / "copernicus"
RAW_GEE = ruta_base / "data" / "raw" / "gee"

INTERIM_CHIRPS = ruta_base / "data" / "interim" / "chirps"
INTERIM_COPERNICUS = ruta_base / "data" / "interim" / "copernicus"
INTERIM_GEE = ruta_base / "data" / "interim" / "gee"

PROCESSED_CHIRPS = ruta_base / "data" / "processed" / "chirps"
PROCESSED_COPERNICUS = ruta_base / "data" / "processed" / "copernicus"
PROCESSED_GEE = ruta_base / "data" / "processed" / "gee"

# Instalación de requirements 
req_path = ruta_base / "requirements.txt"
if req_path.exists():
    print(f"📦 Instalando dependencias desde: {req_path}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(req_path)])
else:
    print("⚠️ No se encontró requirements.txt. Asegúrate de tener las dependencias instaladas.")


Ruta base del proyecto agregada a sys.path: D:\OneDrive - CGIAR\Desktop\climate_data_downloader
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\chirps
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\copernicus
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\chirps
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\copernicus
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\gee
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\copernicus
📁 Carpeta creada/verificada: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\ge

# 🧠 Carga de funciones del proyecto

En esta sección se importan todas las funciones necesarias que han sido previamente desarrolladas en los scripts del proyecto:

- 📥 `download.chirps`  
  Funciones de descarga y validación de datos CHIRPS.

- ☀️ `download.copernicus`  
  Funciones para descargar datos climáticos desde Copernicus (temperatura, radiación, humedad, etc.).

- 🔧 `download.utils`  
  Funciones utilitarias como:
  - Verificación de conexión a Internet
  - Validación de archivos descargados
  - Registro de logs

- 🛠 `download.process`  
  Funciones de procesamiento espacial de los datos, incluyendo:
  - `descomprimir_archivo_gz()` → descomprime archivos `.gz` en formatos `.tif` o `.nc`
  - `recortar_raster()` → recorta un raster con shapefile o bounding box

Estas funciones encapsulan toda la lógica del pipeline para mantener el notebook limpio, modular y reutilizable.


In [2]:
# Importación paquetes requirements
import geopandas as gpd
from tqdm import tqdm

# Importación de funciones de descarga y procesamiento
from download.chirps import descargar_chirps
from download.process import descomprimir_archivo_gz, recortar_raster, descomprimir_archivo_generico
from download.copernicus import download_variable, download_simple_variable, download_humidity, ensure_cdsapi_config
from download.utils import log_event

## 📍 Área de interés y configuración global de fechas

En esta sección se define la configuración **base** que será utilizada por todas las funciones de descarga a lo largo del notebook:

- **Área de interés (AOI)**:  
  Se carga el shapefile del departamento **Valle del Cauca**, ubicado en `data/auxiliary/shapefiles/valle/valle.shp`.  
  A partir de este archivo se calcula automáticamente el **bounding box** necesario para las consultas a la API de Copernicus.

- **Rango de fechas**:  
  Se establece un rango temporal definido por `fecha_inicio` y `fecha_fin`, con un intervalo diario.  
  Este rango será reutilizado para construir las fechas de descarga tanto para **CHIRPS** como para las variables de **Copernicus**.

Estas variables globales permiten mantener un flujo de trabajo coherente y evitar duplicación de parámetros en cada sección del notebook.


In [3]:
# 📍 Área de interés
shapefile_valle = ruta_base / "data" / "auxiliary" / "shapefiles" / "valle" / "valle.shp"
gdf_valle = gpd.read_file(shapefile_valle)
gdf_valle_bounds = gdf_valle.total_bounds  # [xmin, ymin, xmax, ymax]
gdf_valle_bbox = [gdf_valle_bounds[3], gdf_valle_bounds[0], gdf_valle_bounds[1], gdf_valle_bounds[2]]  # [N, W, S, E]

# 📆 Rango de fechas: 2014-01-01 hasta 2024-12-31
fecha_inicio = date(2014, 1, 1)
fecha_fin = date(2014, 2, 5)


# 🌧️ Descarga de datos CHIRPS

En esta sección se realiza la descarga de datos de precipitación del dataset **CHIRPS** (Climate Hazards Group InfraRed Precipitation with Station data).

CHIRPS proporciona datos satelitales de precipitación diarios con cobertura global, ideales para estudios climáticos y agrícolas.

### 🧩 Detalles del flujo:

- Se define un rango de fechas deseado.
- Se construye automáticamente la URL de descarga para cada archivo `.tif.gz`
- Se descarga cada archivo en la carpeta `data/raw/chirps/`
- Luego se descomprime y se guardad en la carpeta `data/interim/chirps/` 
- El resultado recortado se guarda en `data/processed/chirps/`

El proceso incluye verificación de archivos, control de errores, conexión a Internet y logs de eventos.


In [7]:
delta = timedelta(days=1)
fecha = fecha_inicio

while fecha <= fecha_fin:
    anio = fecha.year
    fecha_str = fecha.strftime("%Y-%m-%d")
    nombre_archivo = f"chirps-v2.0.{fecha_str.replace('-', '.')}.tif.gz"

    url = f"https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/{anio}/{nombre_archivo}"
    ruta_gz = RAW_CHIRPS / nombre_archivo
    ruta_tif = INTERIM_CHIRPS / nombre_archivo.replace(".gz", "")
    ruta_clip = PROCESSED_CHIRPS / f"valle_{fecha_str}.tif"

    try:
        print(f"\n📅 Fecha: {fecha_str}")
        print(f"⬇️ URL: {url}")
        print(f"📥 RAW:        {ruta_gz}")
        print(f"🗜️ INTERIM:   {ruta_tif}")
        print(f"📦 PROCESSED:    {ruta_clip}")

        # Paso 1: Descargar
        descargar_chirps(url, ruta_gz)

        # Paso 2: Descomprimir
        descomprimir_archivo_generico(ruta_gz, ruta_tif)

        # Paso 3: Recortar
        recortar_raster(
            ruta_raster=ruta_tif,
            ruta_salida=ruta_clip,
            ruta_shapefile=shapefile_valle
        )
        
    except Exception as e:
        print(f"❌ Error en la fecha {fecha_str}: {e}")

    fecha += delta


📅 Fecha: 2014-01-01
⬇️ URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/2014/chirps-v2.0.2014.01.01.tif.gz
📥 RAW:        D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\chirps\chirps-v2.0.2014.01.01.tif.gz
🗜️ INTERIM:   D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\chirps\chirps-v2.0.2014.01.01.tif
📦 PROCESSED:    D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-01-01.tif


Descargando chirps-v2.0.2014.01.01.tif.gz: 100%|██████████| 2.71M/2.71M [00:00<00:00, 3.38MB/s]


🗜️ Archivo GZIP descomprimido: chirps-v2.0.2014.01.01.tif
Raster recortado guardado en: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-01-01.tif

📅 Fecha: 2014-01-02
⬇️ URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/2014/chirps-v2.0.2014.01.02.tif.gz
📥 RAW:        D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\chirps\chirps-v2.0.2014.01.02.tif.gz
🗜️ INTERIM:   D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\chirps\chirps-v2.0.2014.01.02.tif
📦 PROCESSED:    D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-01-02.tif


Descargando chirps-v2.0.2014.01.02.tif.gz: 100%|██████████| 2.53M/2.53M [00:00<00:00, 3.07MB/s]


🗜️ Archivo GZIP descomprimido: chirps-v2.0.2014.01.02.tif
Raster recortado guardado en: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-01-02.tif

📅 Fecha: 2014-01-03
⬇️ URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/2014/chirps-v2.0.2014.01.03.tif.gz
📥 RAW:        D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\chirps\chirps-v2.0.2014.01.03.tif.gz
🗜️ INTERIM:   D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\chirps\chirps-v2.0.2014.01.03.tif
📦 PROCESSED:    D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-01-03.tif


...

🗜️ Archivo GZIP descomprimido: chirps-v2.0.2014.02.03.tif
Raster recortado guardado en: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-02-03.tif

📅 Fecha: 2014-02-04
⬇️ URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/2014/chirps-v2.0.2014.02.04.tif.gz
📥 RAW:        D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\chirps\chirps-v2.0.2014.02.04.tif.gz
🗜️ INTERIM:   D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\chirps\chirps-v2.0.2014.02.04.tif
📦 PROCESSED:    D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-02-04.tif


Descargando chirps-v2.0.2014.02.04.tif.gz: 100%|██████████| 3.84M/3.84M [00:01<00:00, 2.99MB/s]


🗜️ Archivo GZIP descomprimido: chirps-v2.0.2014.02.04.tif
Raster recortado guardado en: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-02-04.tif

📅 Fecha: 2014-02-05
⬇️ URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/2014/chirps-v2.0.2014.02.05.tif.gz
📥 RAW:        D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\chirps\chirps-v2.0.2014.02.05.tif.gz
🗜️ INTERIM:   D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\chirps\chirps-v2.0.2014.02.05.tif
📦 PROCESSED:    D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-02-05.tif


Descargando chirps-v2.0.2014.02.05.tif.gz: 100%|██████████| 3.76M/3.76M [00:01<00:00, 2.60MB/s]


🗜️ Archivo GZIP descomprimido: chirps-v2.0.2014.02.05.tif
Raster recortado guardado en: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\processed\chirps\valle_2014-02-05.tif


# ☀️ Descarga de variables climáticas desde Copernicus

En esta sección se descarga información climática desde la **API de Copernicus**, específicamente del conjunto de productos `sis-agrometeorological-indicators`.

### 📦 Variables consideradas:

- ☀️ `solar_radiation_flux`
- 💧 `2m_relative_humidity`
- 🌡 `2m_temperature` (máxima, mínima, media)
- 💨 `10m_wind_speed`
- ❄️ `2m_dewpoint_temperature`

Cada variable se descarga en formato `.nc.gz`, se puede descomprimir y luego utilizar en análisis o modelamiento.

### 🗂 Organización de carpetas:

Para mantener los datos organizados, se crean subcarpetas por variable dentro de las carpetas base:

- `data/raw/copernicus/SOLAR_RADIATION_FLUX/`
- `data/interim/copernicus/SOLAR_RADIATION_FLUX/`
- `data/processed/copernicus/SOLAR_RADIATION_FLUX/`

Esto permite manejar múltiples fuentes y variables de forma ordenada y reproducible.


### 🛠  Configuración de la api key:

Se crea el archivo `.cdsapirc` con la api key incluida en el archivo .env, en caso de ya exista el archivo el proceso lo actualiza con la nueva api key suministrada.

In [5]:
# Variables climáticas de Copernicus
copernicus_variables = [
    "solar_radiation_flux",
    "2m_relative_humidity",
    "2m_temperature_max",
    "2m_temperature_min",
    "2m_temperature_mean",
    "2m_dewpoint_temperature",
    "10m_wind_speed"
]

# Diccionarios para almacenar rutas por variable
RAW_COPERNICUS_VARS = {}
INTERIM_COPERNICUS_VARS = {}
PROCESSED_COPERNICUS_VARS = {}

# Crear subcarpetas organizadas por variable dentro de cada etapa
for var in copernicus_variables:
    var_folder = var.upper()

    raw_var_path = RAW_COPERNICUS / var_folder
    interim_var_path = INTERIM_COPERNICUS / var_folder
    processed_var_path = PROCESSED_COPERNICUS / var_folder

    # Crear las carpetas si no existen
    for path in [raw_var_path, interim_var_path, processed_var_path]:
        path.mkdir(parents=True, exist_ok=True)

    # Guardar rutas en diccionarios para uso posterior
    RAW_COPERNICUS_VARS[var] = raw_var_path
    INTERIM_COPERNICUS_VARS[var] = interim_var_path
    PROCESSED_COPERNICUS_VARS[var] = processed_var_path

print("✅ Carpetas para cada variable de Copernicus creadas exitosamente.")


✅ Carpetas para cada variable de Copernicus creadas exitosamente.


In [9]:
# Asegurarse de que el archivo .cdsapirc exista o se genere
ensure_cdsapi_config()

✅ Archivo .cdsapirc creado o reemplazado en: C:\Users\dagudelo\.cdsapirc


## ☀️ Descarga de radiación solar (`solar_radiation_flux`)

En esta sección se descarga la variable **radiación solar** desde la plataforma de Copernicus Climate Data Store (CDS), utilizando el producto:



### 🔍 Detalles técnicos:

- **Variable**: `solar_radiation_flux`
- **Formato de descarga**: `.nc.gz` (NetCDF comprimido)
- **Área de interés**: `valle.shp` (Valle del Cauca)
- **Periodo**: se define en el notebook (año, mes)
- **Frecuencia**: diaria
- **Estadística aplicada**: ninguna (variable simple)

### 📁 Organización de datos:

Los archivos serán almacenados en:

- `data/raw/copernicus/SOLAR_RADIATION_FLUX/` → archivo `.nc.gz`
- `data/interim/copernicus/SOLAR_RADIATION_FLUX/` → archivo `.nc` descomprimido
- `data/processed/copernicus/SOLAR_RADIATION_FLUX/` → (opcional para transformaciones posteriores)

Esta estructura permite trabajar de forma modular con múltiples fuentes y variables sin conflictos de nombres ni rutas.


In [10]:

# ⚙️ Parámetros base
producto = "sis-agrometeorological-indicators"
variable = "solar_radiation_flux"
raw_dir = RAW_COPERNICUS_VARS[variable]
interim_dir = INTERIM_COPERNICUS_VARS[variable]

# 🔁 Construir conjunto (año, mes) desde el rango definido
fechas_unicas = set()
fecha = fecha_inicio
while fecha <= fecha_fin:
    fechas_unicas.add((fecha.year, fecha.month))
    fecha += timedelta(days=1)

# 📥 Descargar y 🗜️ descomprimir por mes
for (anio, mes) in sorted(fechas_unicas):
    nombre_archivo_gz = f"{variable}_{anio}_{mes:02d}.nc.gz"
    nombre_archivo_nc = nombre_archivo_gz.replace(".gz", "")
    ruta_gz = raw_dir / nombre_archivo_gz
    ruta_nc = interim_dir / nombre_archivo_nc

    print(f"\n📥 Descargando {variable} → {anio}-{mes:02d}")
    try:
        download_simple_variable(
            product=producto,
            variable=variable,
            year=anio,
            month=mes,
            area=gdf_valle_bbox,
            output_path=ruta_gz
        )

        print(f"🗜️ Descomprimiendo → {ruta_nc.name}")
        descomprimir_archivo_generico(ruta_gz, ruta_nc)

    except Exception as e:
        print(f"❌ Error en {anio}-{mes:02d}: {e}")




📥 Descargando solar_radiation_flux → 2014-01


2025-04-06 19:39:39,994 WARNING [2025-04-03T00:00:00] System is in degraded status due to issues on the underlying infrastructure. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/data-stores-in-degraded-status/12472)
2025-04-06 19:39:39,996 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-06 19:39:39,996 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-06 19:39:40,505 INFO [2024-02-01T00:00:00] An issue has been identified with version 1.0 & version 1.1 of this dataset. Please see the Known issues table in the Documentation for more information.
2025-04-06 19:39:40,506 INFO Request ID is 5fff4f5b-f501-4b61-b517-2568b075d629


🗜️ Descomprimiendo → solar_radiation_flux_2014_01.nc
🗜️ Archivo ZIP extraído a: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\copernicus\SOLAR_RADIATION_FLUX

📥 Descargando solar_radiation_flux → 2014-02


2025-04-06 19:39:49,936 WARNING [2025-04-03T00:00:00] System is in degraded status due to issues on the underlying infrastructure. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/data-stores-in-degraded-status/12472)
2025-04-06 19:39:49,936 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-06 19:39:49,936 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-06 19:39:50,444 INFO [2024-02-01T00:00:00] An issue has been identified with version 1.0 & version 1.1 of this dataset. Please see the Known issues table in the Documentation for more information.
2025-04-06 19:39:50,446 INFO Request ID is f36dadb7-9afc-4215-aa53-e3ac105055ca


🗜️ Descomprimiendo → solar_radiation_flux_2014_02.nc
🗜️ Archivo ZIP extraído a: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\interim\copernicus\SOLAR_RADIATION_FLUX


## 🌍 Variables descargadas desde Google Earth Engine (GEE)

En esta sección se descargarán variables de teledetección provenientes de **MODIS** utilizando la plataforma Google Earth Engine y el paquete `geemap`.

### 📦 Variables consideradas:

- **NDVI** – Índice de Vegetación de Diferencia Normalizada  
  - Fuente: MODIS  
  - Descripción: Representa la cantidad y vigor de vegetación verde.  
  - Aplicación: Análisis de cobertura vegetal, estrés hídrico, productividad.  

- **NDWI** – Índice de Diferencia Normalizada de Humedad  
  - Fuente: MODIS  
  - Descripción: Estima el contenido de agua en vegetación o cuerpos de agua superficiales.  
  - Aplicación: Monitoreo de humedad del suelo, zonas inundadas o estrés hídrico.

Estas variables se generarán a partir de productos MODIS, utilizando combinaciones específicas de bandas, y se exportarán en formato raster `.tif`, ya recortadas al área de interés definida en el shapefile del **Valle del Cauca**.


## 🔐 Autenticación con Google Earth Engine (solo la primera vez)

Google Earth Engine requiere autenticación la primera vez que se usa.  
Este proceso abrirá una pestaña en el navegador para iniciar sesión con tu cuenta de Google.

Después de autenticarse una vez, el token se guarda localmente y ya no es necesario repetir este paso.


In [4]:
import ee
import geemap
from pathlib import Path

# Ruta donde se guarda el token de autenticación local
credenciales_path = Path.home() / ".config" / "earthengine" / "credentials"

# Autenticar si es necesario
if not credenciales_path.exists():
    print("🔐 No se encontró autenticación previa de Earth Engine. Iniciando login...")
    ee.Authenticate()
else:
    print("✅ Earth Engine ya autenticado.")

# Inicializar sesión
ee.Initialize()
print("🌍 Google Earth Engine inicializado.")


✅ Earth Engine ya autenticado.
🌍 Google Earth Engine inicializado.


In [5]:
# Variables a descargar desde Google Earth Engine
gee_variables = ["NDVI", "NDWI"]

# Diccionarios para rutas por variable
RAW_GEE_VARS = {}
INTERIM_GEE_VARS = {}
PROCESSED_GEE_VARS = {}

# Crear subcarpetas organizadas por variable
for var in gee_variables:
    var_folder = var.upper()

    raw_path = RAW_GEE / var_folder
    interim_path = INTERIM_GEE / var_folder
    processed_path = PROCESSED_GEE / var_folder

    # Crear carpetas si no existen
    for path in [raw_path, interim_path, processed_path]:
        path.mkdir(parents=True, exist_ok=True)

    # Guardar rutas en diccionarios
    RAW_GEE_VARS[var] = raw_path
    INTERIM_GEE_VARS[var] = interim_path
    PROCESSED_GEE_VARS[var] = processed_path

print("✅ Carpetas para NDVI y NDWI (GEE) creadas correctamente.")


✅ Carpetas para NDVI y NDWI (GEE) creadas correctamente.


In [6]:
from download.gee import descargar_ndvi_modis

# 📥 Descarga de NDVI MODIS para el Valle del Cauca
print("🌱 Iniciando descarga de NDVI desde MODIS...")
print(f"🗓️ Rango de fechas: {fecha_inicio.strftime('%Y-%m-%d')} a {fecha_fin.strftime('%Y-%m-%d')}")
print(f"📍 Shapefile AOI: {shapefile_valle}")
print(f"📁 Carpeta de salida: {RAW_GEE_VARS['NDVI']}")

descargar_ndvi_modis(
    fecha_inicio=fecha_inicio.strftime("%Y-%m-%d"),
    fecha_fin=fecha_fin.strftime("%Y-%m-%d"),
    shapefile_path=str(shapefile_valle),
    output_folder=str(RAW_GEE_VARS["NDVI"])
)

🌱 Iniciando descarga de NDVI desde MODIS...
🗓️ Rango de fechas: 2014-01-01 a 2014-02-05
📍 Shapefile AOI: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\auxiliary\shapefiles\valle\valle.shp
📁 Carpeta de salida: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee\NDVI
📦 Imágenes encontradas: 35
Generating URL ...
Please wait ...
Data downloaded to D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee\NDVI\NDVI_MODIS_2014-01-01.tif
✅ Descargado: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee\NDVI\NDVI_MODIS_2014-01-01.tif
Generating URL ...
Please wait ...
Data downloaded to D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee\NDVI\NDVI_MODIS_2014-01-02.tif
✅ Descargado: D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee\NDVI\NDVI_MODIS_2014-01-02.tif
Generating URL ...
Please wait ...
Data downloaded to D:\OneDrive - CGIAR\Desktop\climate_data_downloader\data\raw\gee\NDVI\NDVI_MODIS_2014-01-03.tif
✅ Descargado: